# Instalar dependencias

In [1]:
%pip install openai dotenv gradio 

Note: you may need to restart the kernel to use updated packages.


# Importar librerias

In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import gradio as gr

c:\Users\Estudiante\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Definir las variables de las API KEYS

In [3]:
load_dotenv(override=True)
open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# Conectar los distintos modelos

## Google

In [4]:
#gemini = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
#model_name = "gemini-2.5-flash"

gemini_model = dict(model=OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/"), model_name="gemini-2.5-flash")

response = gemini_model["model"].chat.completions.create(model=gemini_model["model_name"], messages=[
    {
      "role": "user",
      "content": "Que es AI"
    }
  ])
answer = response.choices[0].message.content
answer

'**IA** significa **Inteligencia Artificial**.\n\nEn términos sencillos, la Inteligencia Artificial se refiere a la capacidad de las máquinas o sistemas informáticos para **realizar tareas que normalmente requieren inteligencia humana**. Esto incluye cosas como:\n\n*   **Aprender** de la experiencia (es decir, de los datos).\n*   **Resolver problemas**.\n*   **Comprender** el lenguaje humano (hablado o escrito).\n*   **Reconocer** patrones, objetos o rostros en imágenes y videos.\n*   **Tomar decisiones** basadas en la información.\n*   **Adaptarse** a nuevas situaciones.\n\n**¿Cómo funciona?**\nLa IA se basa en algoritmos complejos y modelos matemáticos que procesan grandes cantidades de datos. Un campo clave dentro de la IA es el **aprendizaje automático (Machine Learning)**, donde los sistemas aprenden de los datos sin ser programados explícitamente para cada tarea. Dentro del aprendizaje automático, está el **aprendizaje profundo (Deep Learning)**, que utiliza redes neuronales arti

## OpenRouterAI

In [5]:
#open_router_ai = OpenAI(
#  base_url="https://openrouter.ai/api/v1",
#  api_key=open_router_api_key,
#)

open_router_model = dict(model=OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=open_router_api_key,
), 
model_name="deepseek/deepseek-r1-0528")

completion = open_router_model["model"].chat.completions.create(
  #extra_headers={
  #  "HTTP-Referer": "MI PAGINA o APP", 
  #  "X-Title": "ANDY CODE",
  #},
  model=open_router_model["model_name"],
  messages=[
    {
      "role": "user",
      "content": "Que es AI"
    }
  ]
)
print(completion.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}

## Groq

In [ ]:
#grog_ai = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
#model_name = "llama-3.3-70b-versatile"

groq_model = dict(model=OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1"), model_name="llama-3.3-70b-versatile")

response = groq_model["model"].chat.completions.create(model=groq_model["model_name"], messages=[
    {
      "role": "user",
      "content": "Que es AI"
    }
  ])
answer = response.choices[0].message.content
answer

# Crear interfaz de chat con gradio

## Funcion de chat

In [ ]:
import uuid


def obtener_valor(dict: dict, key):
    return dict.get(key, "Clave no encontrada")

models = dict(
    gemini=gemini_model,
    #open_router=open_router_model,
    #groq=groq_model
)

chats = {}

def chat(message, history, model, chats_state, current_chat_id):

    # TODO: fix issue where sending a message restores full history from other chats

    try:

        print(f"Modelo seleccionado: {model}")
        print(f"modelo: {obtener_valor(models[model], 'model_name')}")

        # Asegurar que hay una lista para ese chat
        if current_chat_id not in chats_state or chats_state[current_chat_id] is None:
            chats_state[current_chat_id] = []

        # Primero, guardar el history que entró (para no perder mensajes previos)
        chats_state[current_chat_id] = history

        messages = [
            {"role": "system", "content": "Eres un asistente de IA que responde preguntas y ayuda con tareas."}
        ]
        
        for msg in history:
            messages.append({"role": msg["role"], "content": msg["content"]})
        
        messages.append({"role": "user", "content": message})


        resp = obtener_valor(models[model], "model").chat.completions.create(
            model=obtener_valor(models[model], "model_name"),
            messages=messages,
        )

        # Construir nuevo historial actualizado
        new_history = history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": resp}
        ]

        # Guardar nueva historia en estado
        chats_state[current_chat_id] = new_history

        return resp.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return f"Error: {e}"
    
def new_chat(chats_state: dict):
    """Crea un nuevo chat y lo añade al estado"""
    chat_id = str(uuid.uuid4())[:8]  # ID único corto

    while chat_id in chats_state:
        chat_id = str(uuid.uuid4())[:8]

    chats_state[chat_id] = []  # historial vacío

    return chats_state, chat_id, gr.Dropdown(choices=list(chats_state.keys()), value=chat_id), []

def switch_chat(chats_state, chat_id):
    """Cambia entre chats"""
    return chats_state.get(chat_id, [])

def save_message(chats_state, chat_id, history):
    """Guarda el historial en el chat actual"""
    if chat_id:
        chats_state[chat_id] = history
    return chats_state


## Creacion de la interfaz del Chat con Gradio

In [9]:
with gr.Blocks() as app:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Elija su modelo de IA preferido.")
            model_selector = gr.Dropdown(
                choices=list(models.keys()),
                value="gemini",
                label="Modelo de IA"
            )

            selected_chat = gr.Dropdown(choices=[],
                                     value="",
                                     label="Historial de chats"
                                    )
            
            new_chat_btn =gr.Button("➕ Nuevo Chat", size="md")
            
            chat_state = gr.State({})
            current_chat = gr.State("")

        with gr.Column(scale=3):

            current_model = gr.State("gemini")

            model_selector.change(
                fn=lambda model: model,
                inputs=model_selector,
                outputs=current_model
            )

            chatbot = gr.Chatbot(type="messages")

            gr.ChatInterface(chat, 
                             chatbot=chatbot, 
                             additional_inputs=[current_model, chat_state, current_chat], 
                             type="messages", 
                             title="Chat con Modelos de IA"
                            )

            selected_chat.change(
                fn=switch_chat,
                inputs=[chat_state, selected_chat],
                outputs=chatbot
            ).then(
                fn= lambda cid: cid,
                inputs=selected_chat,
                outputs=current_chat
            )

            new_chat_btn.click(fn=new_chat,
                               inputs=[chat_state],
                               outputs=[chat_state, current_chat, selected_chat, chatbot])

            chatbot.change(
                fn=save_message,
                inputs=[chat_state, current_chat, chatbot],
                outputs=chat_state
            )




app.launch()

c:\Users\Estudiante\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\components\dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7909
* To create a public link, set `share=True` in `launch()`.


Modelo seleccionado: gemini
modelo: gemini-2.5-flash
Modelo seleccionado: gemini
modelo: gemini-2.5-flash
Modelo seleccionado: gemini
modelo: gemini-2.5-flash
